# Jupyter Notebooklets Demo

### [@ianhellen](https://twitter.com/ianhellen)
#### Principal Dev - MSTIC, Azure Security

# What are notebooklets?

Collections of notebook cells that implement some useful reusable sequence

## Rationale
- Notebook code can quickly become complex and length:
  - Can obscure the information you are trying to display
  - Can be intimidating to non-developers
- Notebook code cells are not easily re-useable:
  - You can copy and paste but how do you sync changes back to original notebook?
  - Difficult to discover code snippets in notebooks
- Notebook code is often fragile:
  - Often not parameterized
  - Code blocks are frequently dependent on global values assigned earlier
  - Output data is not in any standard format
  - Difficult to test

## Characteristics of Notebooklets
- One or small number of entry points
- Must be paramertizable (e.g. you can supply hostname, IP Address, time range, etc.)
- Can query, process or visualize data (or any combination)
- Typically return a result or package of results for use later in the notebook


---
# Initializing the Notebook
Notebooklets depend on msticpy so we import/initialize this package.

In [2]:
import sys
import os
from IPython.display import display, HTML, Markdown

from msticpy.nbtools.nbinit import init_notebook
init_notebook(namespace=globals());

Processing imports....
Checking configuration....
No errors found.

The following configuration warnings were found: 
 -------------------------------------------------
No AzureCLI section in settings.
Setting options....


---
# Notebooklets in use

## Import the package
- Discovers and imports notebooklet classes/modules

In [3]:
# pip install msticnb

In [4]:
import msticnb as nb

7 notebooklets loaded.


---
## Calling init()
Before using any of the notebooklets you need to initialize the providers.

Providers are the libraries that do the work of fetching data from external sources that are
then used by the notebooklet code.

init() does the following:
- Loads required data providers
- Authenticates to providers if required at startup
- Can supply list of providers to load
- Can pass parameters to each provider (settings loaded from config by default)

In [5]:
nb.init?

Signature:
nb.init(
    query_provider:str='LogAnalytics',
    providers:Union[List[str], NoneType]=None,
    **kwargs,
)
Docstring:
Instantiate an instance of DataProviders.

Parameters
----------
query_provider : str, optional
    DataEnvironment name of the primary query provider.
    You can add addtional query providers by including them
    in the `providers` list.
providers : Optional[List[str]], optional
    A list of provider names, by default "LogAnalytics"

Other Parameters
----------------
kwargs
    You can pass parameters to individual providers using
    the following notation:
    `ProviderName_param_name="param_value"
    Where `ProviderName` is the name of the data provider,
    `param_name` is the parameter name expected by the
    provider and `param_value` is the value to assign to
    `param_name`. `param_value` can be any type.

    Depending on the provider, these parameters (with the
    prefix stripped) are sent to either the constructor or
    `connect` metho

### Available Providers

In [6]:
nb.DataProviders.list_providers()

['LogAnalytics',
 'AzureSentinel',
 'Kusto',
 'AzureSecurityCenter',
 'SecurityGraph',
 'MDATP',
 'LocalData',
 'Splunk',
 'tilookup',
 'geolitelookup',
 'ipstacklookup']

### Default Providers

In [7]:
nb.DataProviders.get_def_providers()

['tilookup', 'geolitelookup']

### Running init, adding ipstacklookup to the default set of providers.
You can also prefix a provider name with "-" to remove it from the default set.

You can also specify an explicit list of providers to override the defaults entirely. E.g
```
nb.init(query_provider="AzureSentinel", providers=["ipstacklookup", "tilookup"])
```

> **Note** you cannot mix the "+"/"-" with un-prefixed provider names.
> Doing this will cause an error to be thrown.
> e.g. <br>
> `nb.init(query_provider="AzureSentinel", providers=["+ipstacklookup", "tilookup"])`
> <br>is illegal.

In [28]:
nb.init(query_provider="AzureSentinel")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank
Loaded providers: LogAnalytics, geolitelookup, tilookup


---
## Using LocalData Provider
The LocalData provider allows you to substitue local files for queries that you normally
make to online data sources such as AzureSentinel.

When we call init() we use the "LocalData_" prefix to pass the "query_paths" and "data_paths"
parameters to the underlying provider.

- Specify a folder where data files are stored with `LocalData_data_paths` (list[str])
- Specify a folder containing query definition files `LocalData_query_paths` (list[str])


In notebooklets queries are available as self.query_provider

In [26]:
nb.init(
    "LocalData", providers=["-tilookup"],
    LocalData_data_paths=["/src/msticnb/tests/testdata"],
    LocalData_query_paths=["/src/msticnb/tests/testdata"],
)

Loaded providers: LocalData, geolitelookup


---
## Notebooklet classes are discovered and imported at load time

Although you can manually initiate a a run to read more notebooklets.

#### The `nblts` attribute exposes notebooklets (niblets?) in a tree structure
- Useful for autocomplete when you more or less know what you're looking for

The top level in the hierarchy is the data environment (e.g. azsent == AzureSentinel). Beneath these the notebooklets are grouped into various categories such as host, network, etc.

In [10]:
print(nb.nblts)

azsent
  account
    AccountSummary (Notebooklet)
  alert
    EnrichAlerts (Notebooklet)
  host
    HostLogonsSummary (Notebooklet)
    HostSummary (Notebooklet)
    WinHostEvents (Notebooklet)
  network
    NetworkFlowSummary (Notebooklet)
template
  TemplateNB (Notebooklet)



Access an individual notebook using this path structure

In [11]:
nb.nblts.azsent.host.HostSummary?

Init signature:
nb.nblts.azsent.host.HostSummary(
    data_providers:Union[<msticnb.data_providers.SingletonDecorator object at 0x00000223E99C63C8>, NoneType]=None,
    **kwargs,
)
Docstring:     
HostSummary Notebooklet class.

Queries and displays information about a host including:

- IP address assignment
- Related alerts
- Related hunting/investigation bookmarks
- Azure subscription/resource data.


Default Options
---------------
- heartbeat: Query Heartbeat table for host information.
- azure_net: Query AzureNetworkAnalytics table for host network topology information.
- alerts: Query any alerts for the host.
- bookmarks: Query any bookmarks for the host.
- azure_api: Query Azure API for VM information.

Other Options
-------------
None
Init docstring:
Intialize a new instance of the notebooklet class.

Parameters
----------
data_providers : DataProviders, Optional
    Optional DataProviders instance to query data.
    Most classes require this.

Raises
------
MsticnbDataProvide

### Notebooklets are exposed in `nb.nb_index`
The values reflect the physical path in which the notebooklets are stored (you can ignore this)

In [12]:
nb.nb_index

{'nblts.azsent.account.AccountSummary': msticnb.nb.azsent.account.account_summary.AccountSummary,
 'nblts.azsent.alert.EnrichAlerts': msticnb.nb.azsent.alert.ti_enrich.EnrichAlerts,
 'nblts.azsent.host.HostLogonsSummary': msticnb.nb.azsent.host.host_logons_summary.HostLogonsSummary,
 'nblts.azsent.host.HostSummary': msticnb.nb.azsent.host.host_summary.HostSummary,
 'nblts.azsent.host.WinHostEvents': msticnb.nb.azsent.host.win_host_events.WinHostEvents,
 'nblts.azsent.network.NetworkFlowSummary': msticnb.nb.azsent.network.network_flow_summary.NetworkFlowSummary,
 'nblts.template.TemplateNB': msticnb.nb.template.nb_template.TemplateNB}

## There is a find function that looks for:
- text or regulate expressions
- searches class docstring
- metadata such as entities supported and options supported

In [13]:
nb.find("host, net.*", full_match=True)

[('HostSummary', msticnb.nb.azsent.host.host_summary.HostSummary),
 ('NetworkFlowSummary',
  msticnb.nb.azsent.network.network_flow_summary.NetworkFlowSummary)]

---
# More detailed (and user-friendly) help in the `show_help()` method

In [ ]:
nb.nblts.azsent.host.HostSummary.show_help()

---
# How are notebooklets used?

## Most require time range parameters

Usually the notebooklet also the ID of the entity that you're running the notebooklet for. For example, a host name, an IP Address, etc.

Some notebooklets process data in the form of a dataframe. Use the `data` parameter to pass this.

> **Note** You can also pass other parameters used by the notebooklet as keyword arguments (`**kwargs`)

In [30]:
time_span = nbwidgets.QueryTime(auto_display=True, units="day", origin_time=pd.to_datetime("2019-02-10"), before=10)

HTML(value='<h4>Set query time boundaries</h4>')

## Run the notebooklet using the `run()` method

>  **Note:** You'll want to assign the return value of `run()` to something or terminate with a semicolon<br>
>  Both the notebooklet and the return `result` class generate displayable output - so you'll get
>  a lot of duplicated output.

In [31]:
host_summary = nb.nblts.azsent.host.HostSummary()
host_sum_rslt = host_summary.run(value="Msticalertswin1", timespan=time_span)

msticpy.common.exceptions.MsticpyAzureConfigError('No AzureCLI section found in configuration settings.')

Getting data from SecurityEvent...


<IPython.core.display.Javascript object>

Getting data from Syslog...


<IPython.core.display.Javascript object>

...

{ 'AdditionalData': {},
  'AzureDetails': { 'ResourceGroup': 'ASIHUNTOMSWORKSPACERG',
                    'ResourceId': '/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourceGroups/ASIHUNTOMSWORKSPACERG/providers/Microsoft.Compute/virtualMachines/MSTICAlertsWin1',
                    'ResourceProvider': 'Microsoft.Compute',
                    'ResourceType': 'virtualMachines',
                    'Solutions': '"security", "changeTracking", "networkMonitoring", "serviceMap", '
                                 '"dnsAnalytics", "securityCenterFree", "securityInsights", '
                                 '"windowsEventForwarding"',
                    'SubscriptionId': '40dcc8bf-0478-4f3b-b275-ed0a94f2c013'},
  'Environment': 'Azure',
  'HostName': 'MSTICAlertsWin1',
  'IPAddress': { 'AdditionalData': {},
                 'Address': '40.76.43.124',
                 'Location': { 'AdditionalData': {},
                               'CountryName': 'United States',
                   

<IPython.core.display.Javascript object>

Loading BokehJS ...

Getting data from Bookmarks...


<IPython.core.display.Javascript object>

## Result classes content can be displayed in the notebook
Use `display(result)` if you want to display the content in the middle of a cell

In [32]:
host_sum_rslt

HostSummaryResult(description='Host summary', timespan=<msticpy.nbtools.nbwidgets.QueryTime object at 0x00000223EB028CF8>, _attribute_desc={'host_entity': ('msticpy.data.nbtools.entities.Host', 'The host entity object contains data about the host such as name, environment, operating system version, IP addresses and Azure VM details. Depending on the type of host, not all of this data may be populated.'), 'related_alerts': ('pd.DataFrame', 'Pandas DataFrame of any alerts recorded for the host within the query time span.'), 'alert_timeline': ('', 'Bokeh time plot of alerts recorded for host.'), 'related_bookmarks': ('pd.DataFrame', 'Pandas DataFrame of any investigation bookmarks relating to the host.'), 'timespan': ('TimeSpan', 'Time span for the queried results data.')}, host_entity=Host(Type=host, HostName=MSTICAlertsWin1, SourceComputerId=263a788b-6526-4cdc-8ed9-d7...), related_alerts=                                 TenantId             TimeGenerated  \
0    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-10 05:38:41+00:00   
1    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-09 23:27:23+00:00   
2    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-09 23:27:25+00:00   
3    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-07 00:58:29+00:00   
4    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-07 00:06:32+00:00   
..                                    ...                       ...   
296  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-05 17:15:18+00:00   
297  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-06 07:28:11+00:00   
298  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-06 07:58:31+00:00   
299  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-06 06:58:31+00:00   
300  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-06 08:58:32+00:00   

                                      AlertDisplayName  \
0       Security incident with shared process detected   
1    Digital currency mining related behavior detected   
2                  Suspiciously named process detected   
3                                    Sample Alert Rule   
4       Potential attempt to bypass AppLocker detected   
..                                                 ...   
296                Suspicious SVCHOST process executed   
297     Security incident with shared process detected   
298                                  Sample Alert Rule   
299                                  Sample Alert Rule   
300                                  Sample Alert Rule   

                                             AlertName Severity  \
0       Security incident with shared process detected     High   
1    Digital currency mining related behavior detected     High   
2                  Suspiciously named process detected     High   
3                                    Sample Alert Rule   Medium   
4       Potential attempt to bypass AppLocker detected     High   
..                                                 ...      ...   
296                Suspicious SVCHOST process executed     High   
297     Security incident with shared process detected     High   
298                                  Sample Alert Rule   Medium   
299                                  Sample Alert Rule   Medium   
300                                  Sample Alert Rule   Medium   

                                                                                             Description  \
0    The incident which started on 2019-02-09 23:26:48 UTC and recently detected on 2019-02-10 05:38:...   
1    Analysis of host data on MSTICALERTSWIN1 detected the execution of a process or command normally...   
2    Analysis of host data on MSTICALERTSWIN1 detected a process whose name is very similar to but di...   
3                                                                                                          
4    Analysis of host data on MSTICALERTSWIN1 detected a potential attempt to bypass AppLocker restri...   
..                                                                                               

---
## Simple Notebooklet browser

In [18]:
nb.browse()

In [23]:
win_host_events = nb.nblts.azsent.host.WinHostEvents()
timespan = TimeSpan(start="2020-05-07 00:10")
win_host_events_rslt = win_host_events.run(value="MSTICAlertsWin1", timespan=timespan)

Getting data from SecurityEvent...


Activity,DWM-1,DWM-2,IUSR,LOCAL SERVICE,MSTICAdmin,MSTICAlertsWin1$,NETWORK SERVICE,No Account,SYSTEM,ian
1100 - The event logging service has shut down.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
4608 - Windows is starting up.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
4616 - The system time was changed.,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4625 - An account failed to log on.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
4634 - An account was logged off.,0.000000,4.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,2.000000
4647 - User initiated logoff.,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4648 - A logon was attempted using explicit credentials.,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000
4672 - Special privileges assigned to new logon.,2.000000,2.000000,1.000000,1.000000,14.000000,0.000000,1.000000,0.000000,60.000000,0.000000
4720 - A user account was created.,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4722 - A user account was enabled.,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Activity,MSTICAdmin
4720 - A user account was created.,2
4722 - A user account was enabled.,2
4724 - An attempt was made to reset an account's password.,4
4726 - A user account was deleted.,2
4728 - A member was added to a security-enabled global group.,2
4729 - A member was removed from a security-enabled global group.,2
4732 - A member was added to a security-enabled local group.,4
4733 - A member was removed from a security-enabled local group.,3
4738 - A user account was changed.,5


Loading BokehJS ...

## Additional operations apart from `run()`
We can use expand events to unpack the `EventData` column for selected EventIDs

In [20]:
win_host_events_rslt.account_events.head(5)

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,PartitionKey,RowKey,StorageAccount,AzureDeploymentID,AzureTableName,AccessList,AccessMask,AccessReason,AccountDomain,AccountExpires,AccountName,AccountSessionIdentifier,...,TargetUserSid,TemplateContent,TemplateDSObjectFQDN,TemplateInternalName,TemplateOID,TemplateSchemaVersion,TemplateVersion,TokenElevationType,TransmittedServices,UserAccountControl,UserParameters,UserPrincipalName,UserWorkstations,VirtualAccount,VendorIds,Workstation,WorkstationName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId,EventProperties
47,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,27df6071-1e81-4e24-934c-dc96667b83ab,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'MemberName': '-', 'MemberSid': 'S-1-5-21-996632719-2361334927-4038480536-1118', 'TargetUserNam..."
48,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4720,4720 - A user account was created.,,,,,,,,,,%%1794,,,...,,,,,,,,,,\t\t%%2080 \t\t%%2082 \t\t%%2084,%%1793,-,%%1793,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,2c09036a-5ca7-4115-9ddf-e9eb49c14247,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'abai$', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-21-996632..."
49,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.183,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4722,4722 - A user account was enabled.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,fefd6761-e431-4cfa-9cd2-c5700f6186df,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'abai$', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-21-996632..."
50,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.183,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Dummy...",4738,4738 - A user account was changed.,,,,,,,,,,%%1794,,,...,,,,,,,,,,\t\t%%2048 \t\t%%2050,-,-,%%1793,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,1d3997a3-9ede-4f9b-877a-eaabc63a3c1e,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'Dummy': '-', 'TargetUserName': 'abai$', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S..."
51,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.183,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.c

In [21]:
win_host_events.expand_events(event_ids=4728).head(5)

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,MemberName,MemberSid,PrivilegeList,SubjectAccount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TargetAccount,TargetDomainName,TargetSid,TargetUserName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId
47,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1118,-,MSTICAlertsWin1\MSTICAdmin,MSTICAlertsWin1,0xbd57571,MSTICAdmin,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,27df6071-1e81-4e24-934c-dc96667b83ab,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
58,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.447,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1119,-,MSTICAlertsWin1\MSTICAdmin,MSTICAlertsWin1,0xbd57571,MSTICAdmin,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,73b0fe4e-9886-43ab-afa6-b43eb7434402,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...


---
# Anatomy of a Notebooklet

# Three sections:
- Results class - what is it going to return
- Notebooklet class - `run()` defines what the notebooklet does
- Code - series of functions that do the actual work

Run the following cell to import the code.

In [ ]:
nb.nblts.template.TemplateNB.import_cell()

---
# More Info

## msticpy
- Documentation - https://msticpy.readthedocs.io
- GitHub - https://github.com/microsoft/msticpy
- PyPI - https://pypi.org/project/msticpy/

## msticnb - Notebooklets
- GitHub - https://github.com/microsoft/msticnb

## Notebooks
- Azure-Sentinel-Notebooks - https://github.com/Azure/Azure-Sentinel-Notebooks
- Binder-able demo - https://github.com/Azure/Azure-Sentinel-Notebooks/tree/master/nbdemo